In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

db=[]
for i in range(0,7):
    #img1 = cv.imread('db/juice_train0.jpg',0)
    img1 = cv2.imread('db/img'+str(i)+'.jpg',0)
    # Initiate ORB detector
    orb = cv2.ORB_create()
    # find the keypoints with ORB
    kp1 = orb.detect(img1,None)

    # compute the descriptors with ORB
    kp1, des1 = orb.compute(img1, kp1)
    db.append([img1,kp1,des1])


In [15]:
#!/usr/bin/env python

import sys
import cv2

def border_penalty(x,y,w,h,img_w,img_h,thres):
    if y<thres:
        return True
    if x<thres:
        return True
    if x+w>img_w-thres:
        return True
    if y+h>img_h-thres:
        return True
    return False
 
if __name__ == '__main__':
    # If image path and f/q is not passed as command
    # line arguments, quit and display help message
     
    # speed-up using multithreads
    cv2.setUseOptimized(True);
    cv2.setNumThreads(4);
 
    # read image
    for i in range(0,15):
        im = cv2.imread('db2/img'+str(i)+'.jpg',1)
        cv2.imshow("Output", im)

        # key press
        cv2.waitKey(0)

        #im = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # resize image
        newHeight = 200
        newWidth = int(im.shape[1]*200/im.shape[0])
        im = cv2.resize(im, (newWidth, newHeight))    

        # create Selective Search Segmentation Object using default parameters
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

        # set input image on which we will run segmentation
        ss.setBaseImage(im)

        # Switch to fast but low recall Selective Search method
        #ss.switchToSelectiveSearchFast()

        # Switch to high recall but slow Selective Search method
        ss.switchToSelectiveSearchQuality()
        # if argument is neither f nor q print help message

        # run selective search segmentation on input image
        rects = ss.process()
        print('Total Number of Region Proposals: {}'.format(len(rects)))

        # number of region proposals to show
        numShowRects = 10
        # increment to increase/decrease total number
        # of reason proposals to be shown
        increment = 10

        # create a copy of original image
        imOut = im.copy()

        # itereate over all the region proposals
        x_mean=0
        y_mean=0
        w_cum=0
        h_cum=0
        
        x_b=0
        y_b=0
        w_b=0
        h_b=0
        a_b=0
         
        for i, rect in enumerate(rects):
            # draw rectangle for region proposal till numShowRects
            if (i < numShowRects):
                x, y, w, h = rect
                cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
                
                if (w<(2*newWidth/3) and h<(2*newHeight/3) and not border_penalty(x,y,w,h,newWidth,newHeight,5)):
                    x_mean+=x*w
                    y_mean+=y*h
                    w_cum+=w
                    h_cum+=h
                
                if(w*h>a_b and w<(2*newWidth/3) and h<(2*newHeight/3) and not border_penalty(x,y,w,h,newWidth,newHeight,5)):
                    x_b=x
                    y_b=y
                    w_b=w
                    h_b=h
                    a_b=w*h
                    
            else:
                break
        x_mean=(x_mean/w_cum)
        y_mean=(y_mean/h_cum)
        
        cv2.rectangle(imOut, (int(x_mean-50), int(y_mean-50)), (int(x_mean-50)+100, int(y_mean-50)+100), (255, 255, 0), 1, cv2.LINE_AA)
        
        cv2.rectangle(imOut, (x_b, y_b), (x_b+w_b, y_b+h_b), (255, 0, 0), 1, cv2.LINE_AA)
        # show output
        cv2.imshow("Output", imOut)
        cv2.waitKey()

    # record key press
    k = cv2.waitKey(0) & 0xFF

    # m is pressed
    if k == 109:
        # increase total number of rectangles to show by increment
        numShowRects += increment
    # l is pressed
    elif k == 108 and numShowRects > increment:
        # decrease total number of rectangles to show by increment
        numShowRects -= increment
    # q is pressed
    elif k == 113:
        print('break')
    # close image show window
    cv2.destroyAllWindows()

Total Number of Region Proposals: 769
Total Number of Region Proposals: 1117
Total Number of Region Proposals: 1163
Total Number of Region Proposals: 1258
Total Number of Region Proposals: 1335
Total Number of Region Proposals: 1170
Total Number of Region Proposals: 671
Total Number of Region Proposals: 681
Total Number of Region Proposals: 771
Total Number of Region Proposals: 954
Total Number of Region Proposals: 974
Total Number of Region Proposals: 1379
Total Number of Region Proposals: 1387
Total Number of Region Proposals: 1053
Total Number of Region Proposals: 1112


In [18]:
bg=[]
mean= cv2.imread('db2/bg0.jpg',0)
mean = np.float32(mean)
imgs_n=2

#res = np.float32(out)
#scaling, you will have to do separately:

#res = res*scaling_factor

for i in range(1,imgs_n):
    img = cv2.imread('db2/bg'+str(i)+'.jpg',0)
    # Initiate ORB detector
    cv2.imshow("background", np.uint8(mean))
    cv2.waitKey()
    print(cv2.mean(mean))
    mean+=img
    bg.append(img)
    print(cv2.mean(mean))
    

cv2.imshow("background2", mean)
cv2.waitKey()
mean=np.uint8(mean)
cv2.imshow("background3", mean)
cv2.waitKey()

cv2.destroyAllWindows()


(124.20424370659723, 0.0, 0.0, 0.0)
(248.53575737847223, 0.0, 0.0, 0.0)


In [35]:
bg= cv2.imread('db2/bg0.jpg',0)
#gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

img_rgb = cv2.imread('db2/img1.jpg',1)
img=cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

diff=img-bg
cv2.imshow("diff", diff)
cv2.waitKey()

threshold=100

mask = cv2.inRange(diff, threshold, 255)
cv2.imshow("mask", mask)
cv2.waitKey()

img_filt = cv2.bitwise_and(img_rgb, img_rgb, mask= mask)

#cv2.imshow("filtered", img_filt)
cv2.imwrite("demos_imgs/img"+str(i)+".jpg", img_filt)
cv2.waitKey()



-1

In [30]:
def bg_delete(img_rgb,bg_rgb,threshold=100):
    img=cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    bg=cv2.cvtColor(bg_rgb, cv2.COLOR_BGR2GRAY)

    diff=img-bg
    mask = cv2.inRange(diff, threshold, 255)

    img_filt = cv2.bitwise_and(img_rgb, img_rgb, mask= mask)

    return img_filt

bg= cv2.imread('db2/bg0.jpg',1)
img_rgb = cv2.imread('db2/img1.jpg',1)
imgOut=bg_delete(img_rgb,bg)

cv2.imshow("filtered", imgOut)
cv2.waitKey()


-1